### Useful functions within the `aws_tools` module

The `exo_bespin.aws.aws_tools` module provides functions that enables users to use create, start, stop, and interact with AWS EC2 instances.  The relevant functions are:

- `get_config()` - Returns a python dictionary containing the key/value pairs found in the `aws_config.json` file.  This includes the path to the SSH key file, and the `ec2_id`, which points to a specific EC2 instance ID or EC2 launch template ID.
- `start_ec2(ssh_file, ec2_id)` - Starts an EC2 instance for the given `ec2_id`, which can point to a specific EC2 instance ID or an EC2 launch template ID.  Returns `instance`, which is the `boto3` AWS EC2 instance object, `key`, which is the `paramiko.rsakey.RSAKey` object, and `client`, which is the `paramiko.client.SSHClient` object.  `instance`, `key`, and `client` are needed in other functions of `aws_tools` to interact with the EC2 instance.
- `stop_ec2(ec2_id, instance)` - Same as above, only for terminating/stopping the given instance instead of creating/starting it.  If an EC2 instance ID is passed with `ec2_id`, the instance is stopped.  However, if an EC2 launch template ID is passed, the instance is terminated.
- `transfer_to_ec2(instance, key, client, filename)` - Transfers the file given by `filename` to the given EC2 instance.
- `transfer_from_ec2(instance, key, client, filename)` - The same as above, only transfers the given file from the EC2 instance to the local machine.

In [1]:
from exo_bespin.aws import aws_tools

*First, we retrieve the settings from the `aws_config.json` file, which are ultimately needed to interact with EC2:*

In [2]:
settings = aws_tools.get_config()
ssh_file = settings['ssh_file']
ec2_id = settings['ec2_id']

*Next, we use `start_ec2` to create and/or start an EC2 instance.  To create an EC2 instance from scratch, use `ec2_id` to pass an EC2 launch template ID.  To start an existing EC2 instance, use `ec2_id` to pass an EC2 instance ID:*

In [3]:
instance, key, client = aws_tools.start_ec2(ssh_file, ec2_id)

*Please note that it will take a few seconds to start an existing EC2 instance, and take a few minutes to create a new EC2 instance and build the `exo-bespin` software environment.  Users are encourage to use python's `time.sleep()` to make sure that software that uses the EC2 instance does not get invoked too soon.*

*For this example, we will assume the user wishes to run some software on the EC2 instance.  We will use the following simple example module:*

In [4]:
!cat data/ec2_example.py

if __name__ == '__main__':

    # Open up some input file
    with open('input.txt') as f:
        data = f.readlines()
    print(data)

    # Save some results
    with open('results.dat', 'w') as f:
        f.write('These are my results!')

*Note that this example requires an input file (`input.txt`).  We can provide that input file by transfering a copy of it to the EC2 instance using the `transfer_to_ec2` function.  With this, the input file is copied to the home directory of the EC2 instance (i.e. `/home/ec2-user/input.txt`):*

In [5]:
aws_tools.transfer_to_ec2(instance, key, client, 'data/input.txt')

*Now we can run the module of interest:*

In [6]:
command = './exo_bespin/exo_bespin/aws/exo_bespin-env-init.sh python exo_bespin/notebooks/data/ec2_example.py'
client.connect(hostname=instance.public_dns_name, username='ec2-user', pkey=key)
stdin, stdout, stderr = client.exec_command(command)
output = stdout.read()
errors = stderr.read()

# Make output a more readable
output = output.decode("utf-8")
output = output.replace('\t', '  ').replace('\r', '').replace("\'", "").split('\n')
for line in output:
    print(line)

no change     /home/ec2-user/miniconda3/condabin/conda
no change     /home/ec2-user/miniconda3/bin/conda
no change     /home/ec2-user/miniconda3/bin/conda-env
no change     /home/ec2-user/miniconda3/bin/activate
no change     /home/ec2-user/miniconda3/bin/deactivate
no change     /home/ec2-user/miniconda3/etc/profile.d/conda.sh
no change     /home/ec2-user/miniconda3/etc/fish/conf.d/conda.fish
no change     /home/ec2-user/miniconda3/shell/condabin/Conda.psm1
no change     /home/ec2-user/miniconda3/shell/condabin/conda-hook.ps1
no change     /home/ec2-user/miniconda3/lib/python3.8/site-packages/xontrib/conda.xsh
no change     /home/ec2-user/miniconda3/etc/profile.d/conda.csh
modified      /home/ec2-user/.bashrc

==> For changes to take effect, close and re-open your current shell. <==

[Hello World!]



*Note that the example produces a results file.  We can retrieve this results file from the EC2 instance using the `transfer_from_ec2` function.  With this, the results file is copied to the current working directory of the local machine:*

In [7]:
aws_tools.transfer_from_ec2(instance, key, client, 'results.dat')
!ls results.dat

results.dat


*Lastly, to avoid accruing costs, we can stop the EC2 instance using the `stop_ec2` function:*

In [8]:
aws_tools.stop_ec2(ec2_id, instance)